<h1><b>[XII - 1NN]</b></h1>

This is a "1-NN", where only the neighbors that are at `distance 0` (which mean data that are already in the training set) are computed.

Other data are assigned to class `10`

The result will be save in `data/1-NN.csv`

Note that the original `data/1-NN.csv` is already available.

___________________________

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

In [3]:
import time
start = time.time()

train_df_geojson = gpd.read_file('data/train.geojson', engine="pyogrio")
test_df_geojson = gpd.read_file('data/test.geojson', engine="pyogrio")

end = time.time()
print("Reading time:", end - start,"s")

Reading time: 115.6860761642456 s


In [4]:
train_x = train_df_geojson["geometry"].centroid.x
train_y = train_df_geojson["geometry"].centroid.y
train_area = train_df_geojson["geometry"].area

/tmp/ipykernel_17/667961604.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  train_x = train_df_geojson["geometry"].centroid.x
/tmp/ipykernel_17/667961604.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  train_y = train_df_geojson["geometry"].centroid.y
/tmp/ipykernel_17/667961604.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  train_area = train_df_geojson["geometry"].area


In [5]:
test_x = test_df_geojson["geometry"].centroid.x
test_y = test_df_geojson["geometry"].centroid.y
test_area = test_df_geojson["geometry"].area

/tmp/ipykernel_17/461148666.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  test_x = test_df_geojson["geometry"].centroid.x
/tmp/ipykernel_17/461148666.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  test_y = test_df_geojson["geometry"].centroid.y
/tmp/ipykernel_17/461148666.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  test_area = test_df_geojson["geometry"].area


In [6]:
train_infos = np.stack([train_x, train_y, train_area], axis=1)

In [7]:
test_infos = np.stack([test_x, test_y, test_area], axis=1)

In [8]:
from sklearn.neighbors import NearestNeighbors

nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(train_infos)

In [9]:
dists, indices = nbrs.radius_neighbors(test_infos, radius=0.00000000001)

In [10]:
change_type_map = {'Demolition': 0, 'Road': 1, 'Residential': 2, 'Commercial': 3, 'Industrial': 4,
       'Mega Projects': 5}

In [11]:
predicts = []
for iTest in range(len(test_df_geojson)):
    if len(dists[iTest]) == 0:
        predicts.append(10)
        continue
    if len(dists[iTest]) > 1:
        predicts.append(10)
        print(">> Warning : weird", iTest)
        print(dists[iTest])
        print(indices[iTest])
        continue
    indiceTrain = indices[iTest][0]
    predicts.append(change_type_map[train_df_geojson.iloc[indiceTrain]["change_type"]])

>> Warning : weird 11378
[0. 0.]
[22037 22038]
>> Warning : weird 11379
[0. 0.]
[22037 22038]
>> Warning : weird 34121
[0. 0.]
[32164 32165]
>> Warning : weird 34122
[0. 0.]
[32164 32165]
>> Warning : weird 43526
[0. 0.]
[223367 223368]
>> Warning : weird 43527
[0. 0.]
[223367 223368]
>> Warning : weird 56257
[0. 0.]
[241110 241109]
>> Warning : weird 56258
[0. 0.]
[241110 241109]
>> Warning : weird 59736
[0. 0.]
[12246 12247]
>> Warning : weird 59737
[0. 0.]
[12246 12247]
>> Warning : weird 65591
[0. 0.]
[168213 168212]
>> Warning : weird 65592
[0. 0.]
[168213 168212]
>> Warning : weird 71452
[0. 0.]
[248723 248724]
>> Warning : weird 71453
[0. 0.]
[248723 248724]
>> Warning : weird 88202
[0. 0.]
[161412 161413]
>> Warning : weird 88203
[0. 0.]
[161412 161413]
>> Warning : weird 100044
[0. 0.]
[57885 57886]
>> Warning : weird 100045
[0. 0.]
[57885 57886]


In [12]:
pred_df = pd.DataFrame(predicts, columns=['change_type'])
pred_df.to_csv("1-NN.csv", index=True, index_label='Id')

In [14]:
len(pred_df[pred_df["change_type"] != 10])

88515